<a href="https://colab.research.google.com/github/VitalyGladyshev/gb_nlp/blob/main/HW_03_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ДЗ 03 NLP

In [1]:
import numpy as np
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
path_nlp = "/content/gdrive/My Drive/Colab Notebooks/NLP/"

train_cor = pd.read_pickle(path_nlp + "train_cor.pkl")
test_cor = pd.read_pickle(path_nlp + "test_cor.pkl")

test_cor['label'] = -1

data = train_cor.copy()
data = data.append(test_cor, ignore_index=True)
data.tail(2)

,id,label,tweet,clean_tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
49157,49158,-1,"happy, at work conference: right mindset leads...",happy at work conference right mindset leads t...,"[happy, at, work, conference, right, mindset, ...","[happy, work, conference, right, mindset, lead...","{cultureofdevelop, right, confer, happi, lead,...","{right, happy, lead, organizations, conference..."
49158,49159,-1,"my song ""so glad"" free download! #shoegaze ...",my song so glad free download shoegaze newmusi...,"[my, song, so, glad, free, download, shoegaze,...","[song, glad, free, download, shoegaze, newmusi...","{glad, free, song, download, newmus, newsong, ...","{glad, free, song, download, newmusic, newsong..."


## Задание 1

Используя библиотеку Spacy, вывести ТОП-20 популярных NER в combine_df датасете. Какой тип NER (ORG, GPE, PERSON и тд) оказался самым популярным?  (Учтите, что max_word_limit_spacy для Spacy = 1000000)
<br><br>С помощью Spacy выяснить: какие персоны и организации самые обсуждаемые в train и test датасетах? вывести ТОП-20 самых популярных. Действительно ли в топ вошли только персоны и организации или есть мусор? 
<br><br>Повторим шаги из заданий 1 и 2, используя библиотеку nltk.

In [23]:
# !python -m spacy download en_core_web_sm

In [4]:
!python -m spacy download en_core_web_md

     |████████████████████████████████| 96.4MB 1.4MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.5-cp36-none-any.whl size=98051305 sha256=589cce1d4289cdc85e9b5b121f1692617d50fac06038f5e276b3b23dc92b8c37
  Stored in directory: /tmp/pip-ephem-wheel-cache-oa8qe2xh/wheels/df/94/ad/f5cf59224cea6b5686ac4fd1ad19c8a07bc026e13c36502d81
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [5]:
import spacy
from spacy import displacy
import en_core_web_md

In [6]:
nlp = en_core_web_md.load()

In [10]:
doc_t = nlp("Nasa administrator Jim Bridenstine says at the moment of launch, he was praying.")

In [11]:
displacy.render(doc_t, jupyter=True, style='ent')

In [67]:
data['clean_tweet'].values

array(['when father is dysfunctional and is so selfish he drags his kids into his dysfunction run',
       'thanks for lyft credit cannot use cause they do not offer wheelchair vans in pdx disapointed getthanked',
       'bihday your majesty', ...,
       'hillary campaigned today in ohioomg amp used words like assetsampliability never once did clinton say theeword radicalization',
       'happy at work conference right mindset leads to cultureofdevelopment organizations work mindset',
       'my song so glad free download shoegaze newmusic newsong'],
      dtype=object)

In [68]:
str_tweet = " ".join(sent for sent in data['clean_tweet'].values)

In [69]:
print(str_tweet[:1000])

when father is dysfunctional and is so selfish he drags his kids into his dysfunction run thanks for lyft credit cannot use cause they do not offer wheelchair vans in pdx disapointed getthanked bihday your majesty model love you take with you all the time in ur factsguide society now motivation huge fan fare and big talking before they leave chaos and pay disputes when they get there allshowandnogo camping tomorrow danny the next school year is the year for exams cannot think about that school exams hate imagine actorslife revolutionschool girl we won love the land allin cavs champions cleveland clevelandcavaliers welcome here am it has it is so gr ireland consumer price index mom climbed from previous to in may blog silver gold forex we are so selfish orlando standwithorlando pulseshooting orlandoshooting biggerproblems selfish heabreaking values love get to see my daddy today days gettingfed cnn calls michigan middle school build the wall chant tcot no comment in australia opkillingb

In [70]:
nlp = en_core_web_md.load()
doc_tweet = nlp(str_tweet[:1000000])

In [74]:
displacy.render(doc_tweet[:200], jupyter=True, style='ent')

In [34]:
for token in doc_tweet[:20]:
    print(token.text, token.lemma_, token.pos_, token.is_stop)

when when ADV True
father father NOUN False
is be AUX True
dysfunctional dysfunctional ADJ False
and and CCONJ True
is be AUX True
so so ADV True
selfish selfish ADJ False
he -PRON- PRON True
drags drag VERB False
his -PRON- DET True
kids kid NOUN False
into into ADP True
his -PRON- DET True
dysfunction dysfunction NOUN False
run run NOUN False
. . PUNCT False
thanks thank NOUN False
for for ADP True
lyft lyft PROPN False


In [96]:
for ent in doc_tweet.ents[:10]:
    print(ent.text, ent.label_)

lyft credit ORG
pdx GPE
getthanked bihday PERSON
ur factsguide society FAC
tomorrow DATE
danny PERSON
the next school year DATE
the year DATE
actorslife revolutionschool PERSON
cavs champions ORG


In [38]:
type(doc_tweet.ents), len(doc_tweet.ents)

(tuple, 15694)

In [39]:
doc_tweet.ents[:20]

(lyft credit,
 pdx,
 getthanked,
 bihday,
 tomorrow,
 danny,
 the next school year,
 the year,
 actorslife revolutionschool,
 cavs champions,
 ireland,
 orlando standwithorlando pulseshooting orlandoshooting biggerproblems,
 heabreaking,
 today days,
 gettingfed,
 cnn,
 michigan,
 tcot,
 australia,
 seashepherd helpcovedolphins)

In [55]:
type(doc_tweet.ents[0])

spacy.tokens.span.Span

In [61]:
str(doc_tweet.ents[0])

'lyft credit'

In [62]:
ner_list = [str(ent) for ent in doc_tweet.ents]

In [63]:
ner_list[:10]

['lyft credit',
 'pdx',
 'getthanked',
 'bihday',
 'tomorrow',
 'danny',
 'the next school year',
 'the year',
 'actorslife revolutionschool',
 'cavs champions']

In [78]:
vocab = {}
for token in doc_tweet:
    # print(f"token {token.text} type {type(token.text)}")
    # break
    if token.text in ner_list:
        if token.text not in vocab.keys():   # and ():
            vocab[token.text] = 1
        else:
            vocab[token.text] += 1

In [93]:
print([v  for i, v in enumerate(vocab) if i<20])

['lyft', 'pdx', 'getthanked', 'bihday', 'factsguide', 'tomorrow', 'danny', 'year', 'actorslife', 'cavs', 'cleveland', 'am', 'ireland', 'orlando', 'pulseshooting', 'orlandoshooting', 'heabreaking', 'today', 'days', 'gettingfed']


In [76]:
sorted(vocab.items(), key=lambda x: x[1], reverse=True)[:20]

[('am', 1193),
 ('day', 956),
 ('today', 474),
 ('bihday', 365),
 ('one', 358),
 ('weekend', 281),
 ('summer', 252),
 ('us', 250),
 ('great', 233),
 ('days', 225),
 ('bull', 217),
 ('first', 217),
 ('friday', 202),
 ('tomorrow', 194),
 ('morning', 189),
 ('fathersday', 188),
 ('happiness', 184),
 ('week', 180),
 ('night', 168),
 ('orlando', 160)]

## Задание 2

Используя библиотеку nltk, вывести ТОП-20 популярных NER в combine_df датасете. Какой тип NER (ORG, GPE, PERSON и тд) оказался самым популярным? Для данного задания используем ограничение на количество символов во входном датасете (max_word_limit_spacy = 1000000), чтобы иметь возможность сравнить результаты работы Spacy и nltk. Обратите внимание, что nltk чувствителен к регистру.
<br><br>С помощью nltk выяснить: какие персоны и организации самые обсуждаемые в train и test датасетах? вывести ТОП-20 самых популярных. Действительно ли в топ вошли только персоны и организации или есть мусор?


In [7]:
import nltk

In [8]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [9]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [10]:
nltk.download('maxent_ne_chunker')

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.


True

In [11]:
nltk.download('words')

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [12]:
str_tweet = " ".join(sent for sent in data['clean_tweet'].values)

In [17]:
nltk.pos_tag(nltk.word_tokenize(str_tweet[:100].upper()))

[('WHEN', 'WRB'),
 ('FATHER', 'NNP'),
 ('IS', 'NNP'),
 ('DYSFUNCTIONAL', 'NNP'),
 ('AND', 'NNP'),
 ('IS', 'NNP'),
 ('SO', 'NNP'),
 ('SELFISH', 'NNP'),
 ('HE', 'NNP'),
 ('DRAGS', 'NNP'),
 ('HIS', 'NNP'),
 ('KIDS', 'NNP'),
 ('INTO', 'NNP'),
 ('HIS', 'NNP'),
 ('DYSFUNCTION', 'NNP'),
 ('RUN', 'NNP'),
 ('THANKS', 'NNP'),
 ('FOR', 'NNP')]

In [21]:
{(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(str_tweet[:1000].upper()))) if hasattr(chunk, 'label') }

{('ABOUT', 'ORGANIZATION'),
 ('CNN', 'ORGANIZATION'),
 ('CREDIT', 'ORGANIZATION'),
 ('EXAMS', 'ORGANIZATION'),
 ('FACTSGUIDE', 'ORGANIZATION'),
 ('FARE', 'ORGANIZATION'),
 ('FATHER', 'ORGANIZATION'),
 ('GOLD', 'ORGANIZATION'),
 ('HATE', 'ORGANIZATION'),
 ('INTO', 'ORGANIZATION'),
 ('IRELAND', 'ORGANIZATION'),
 ('LAND', 'ORGANIZATION'),
 ('MICHIGAN', 'ORGANIZATION'),
 ('NEXT', 'ORGANIZATION'),
 ('SCHOOL', 'ORGANIZATION'),
 ('THE', 'ORGANIZATION'),
 ('TIME', 'ORGANIZATION'),
 ('USE', 'ORGANIZATION'),
 ('WALL', 'ORGANIZATION'),
 ('WITH', 'ORGANIZATION'),
 ('YEAR FOR', 'ORGANIZATION')}

In [22]:
for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(str_tweet[:200].upper()))):
    if hasattr(chunk, 'label'):
        print(chunk)

(ORGANIZATION FATHER/NNP)
(ORGANIZATION INTO/NNP)
(ORGANIZATION CREDIT/NNP)
(ORGANIZATION USE/NNP)


## Пока не понял как заставить NLTK разбирать текст, который изначально в нижнем регистре. Если всё перевести в верхний, то появляется много ошибочных отметок 'ORGANIZATION', что тоже не вариант.

## Задание 3

Какая из библиотек по вашему лучше отработала? Сравните качество полученных most_common NER и количество распознаных NER.

## Выберем другой текст

In [145]:
import requests
from bs4 import BeautifulSoup
import re

def url_to_string(url):
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html, 'html5lib')
    for script in soup(["script", "style", 'aside']):
        script.extract()
    return " ".join(re.split(r'[\n\t]+', soup.get_text()))

document = url_to_string('https://en.wikipedia.org/wiki/Russia')

In [146]:
len(document), document[:150]

(236210,
 ' Russia - Wikipedia Russia From Wikipedia, the free encyclopedia Jump to navigation Jump to search This article is about the Russian Federation. For o')

### Spacy

In [147]:
nlp = en_core_web_md.load()
doc = nlp(document)

In [148]:
displacy.render(doc[:200], jupyter=True, style='ent')

In [149]:
ner_list = [str(ent) for ent in doc.ents]

In [150]:
vocab = {}
for token in doc:
    if token.text in ner_list:
        if token.text not in vocab.keys():
            vocab[token.text] = 1
        else:
            vocab[token.text] += 1

In [151]:
sorted(vocab.items(), key=lambda x: x[1], reverse=True)[:20]

[('Russia', 544),
 ('Russian', 481),
 ('Soviet', 152),
 ('1', 103),
 ('2014', 90),
 ('December', 86),
 ('Moscow', 78),
 ('27', 67),
 ('April', 64),
 ('2017', 64),
 ('March', 61),
 ('first', 61),
 ('2010', 59),
 ('2007', 58),
 ('January', 57),
 ('2008', 57),
 ('2015', 56),
 ('May', 54),
 ('3', 53),
 ('ISBN', 53)]

### NLTK

In [158]:
for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(document[:1000]))):
    if hasattr(chunk, 'label'):
        print(chunk)

(GPE Russia/NNP)
(PERSON Wikipedia/NNP Russia/NNP)
(PERSON Jump/NNP)
(PERSON Jump/NNP)
(GPE Russian/JJ)
(GPE Russia/NNP)
(GPE Eastern/NNP Europe/NNP)
(GPE Northern/NNP Asia/NNP)
(GPE Russian/JJ)
(ORGANIZATION FederationРоссийская/NNP Федерация/NNP)
(GPE Russian/NNP)
(PERSON Rossiyskaya/NNP Federatsiya/NNP Flag/NNP Coat/NNP)
(PERSON Gosudarstvennyy/NNP)
(PERSON Rossiyskoy/NNP Federatsii/NNP)
(GPE Russian/JJ)
(GPE Russia/NNP)
(GSP Crimea/NNP)
(ORGANIZATION languageRussian/JJ)
(ORGANIZATION languagesSee/NN Languages/NNP)
(GSP RussiaEthnic/NNP)


In [163]:
ner_dict = {' '.join(c[0] for c in chunk): chunk.label() for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(document))) if hasattr(chunk, 'label') }

In [165]:
print([v  for i, v in enumerate(ner_dict) if i<20])

['Russia', 'Wikipedia Russia', 'Jump', 'Russian', 'Eastern Europe', 'Northern Asia', 'FederationРоссийская Федерация', 'Rossiyskaya Federatsiya Flag Coat', 'Gosudarstvennyy', 'Rossiyskoy Federatsii', 'Crimea', 'languageRussian', 'languagesSee Languages', 'RussiaEthnic', 'Islam2', 'Vladimir', 'Mikhail', 'theFederation Council Valentina', 'theState Duma Vyacheslav', 'Vyacheslav Lebedev']


In [169]:
ner_list_nltk = [' '.join(c[0] for c in chunk) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(document))) if hasattr(chunk, 'label')]

In [170]:
ner_list_nltk[:10]

['Russia',
 'Wikipedia Russia',
 'Jump',
 'Jump',
 'Russian',
 'Russia',
 'Eastern Europe',
 'Northern Asia',
 'Russian',
 'FederationРоссийская Федерация']

In [171]:
vocab_nltk = {}
for token in doc:
    if token.text in ner_list_nltk:
        if token.text not in vocab_nltk.keys():
            vocab_nltk[token.text] = 1
        else:
            vocab_nltk[token.text] += 1

In [172]:
sorted(vocab_nltk.items(), key=lambda x: x[1], reverse=True)[:20]

[('Russia', 544),
 ('Russian', 481),
 ('Retrieved', 240),
 ('Soviet', 152),
 ('World', 85),
 ('Moscow', 78),
 ('Union', 63),
 ('United', 63),
 ('ISBN', 53),
 ('Europe', 52),
 ('Republic', 50),
 ('Federation', 49),
 ('China', 48),
 ('Main', 45),
 ('Ukraine', 41),
 ('Putin', 41),
 ('Vladimir', 39),
 ('Orthodox', 39),
 ('PDF', 39),
 ('Sea', 38)]

### Выводы:

- Spacy не требователен к регистру. Реагирует на числительные и даты
- NLTK не удалось разметить текст в нижнем регистре. Определил имена собственные

Каждый из них необходимо применять в зависимости от исходных данных и задач